Ablauf:
Fiel "AQM_import_API"
1. Import Data via API from Refinitiv to Jupiter Notebook
    - 10 Commodities 10 Years
    - Download monthly data for at least 10 asset prices and at least 10 macroeconomic variables, across countries

2. Create DataFrame, check Data and clean Data if needed:
    - N/A
    - Sprünge?
    - Format?
3. Save clean Data in SQL Lite in DB


Zuerst den Code für 1 Kombination schreiben. Danach ein Loop generieren.

File "AQM_Analyse":
1. Import the clean Data from SQL DB into Notebook
2. Prepare Data for Analysis
3. Create Dataframe to save the Results
4. Create Loop, which executes multivariate OLS regressions using all combinations of 1 and 10 explanatory macroeconomic variables 
5. Save the Results from the Loop into the "Result"-Dataframe
    For each Regression calculate:
        Autocorrelation
        heteroskedasticity
        multicollinearity
6. Export Results to SQL Lite into Table "Results"

Todo:
- Direkt von API in Dataframe und danach in SQL ohne Excel
- Excel workaround sauber fertig machen

In [1]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


from datetime import datetime
import time
import datetime

import eikon as ek
creds = 'd505f265d9bd4c0cb7df44481ece31a0ad8d1974'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
#API Key Laurin:
ek.set_app_key(creds)


2022-04-04 11:26:52,996 P[27992] [MainThread 25680] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2022-04-04 11:26:52,998 P[27992] [MainThread 25680] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL missing either an 'http://' or 'https://' protocol.")
2022-04-04 11:26:52,999 P[27992] [MainThread 25680] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL missing either an 'http://' or 'https://' protocol.")
2022-04-04 11:26:53,000 P[27992] [MainThread 25680] Port number was not identified, cannot send any request


In [ ]:
#Define the Future Contracts
Futures = ['CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]

In [ ]:
data_all = pd.DataFrame()
try:
    ek.set_app_key('f5a341b614d945b9b93a71fe53348ba119d11e99')
    itemName=Futures
    dateStart="2012-01-01"
    dateEnd="2022-01-01"

    #Create Loop to inport all Futures:
    for y in range(0,len(Futures)):
        data_input = ek.get_timeseries(Futures[y],fields=["Close"], start_date = dateStart , 
                                    end_date = dateEnd , interval='monthly', corax = 'adjusted' )
        data_input['Instrument'] = Futures[y]
        data_all = data_all.append(data_input)
except ek.EikonError as ex:
    print('Eikon Error')
    print(ex)
except Exception as ex:
    print(ex)

data_all

In [ ]:
data_all['Return'] = np.log(data_all.CLOSE) - np.log(data_all.CLOSE.shift(1))
data_all = data_all.reset_index()
data_all['Date'] = pd.to_datetime(data_all['Date']).dt.date
liste_SQL = data_all.values.tolist()
data_all

Save Data in SQL Database (DB)

In [ ]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

In [ ]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS COMMODITIES_RETURNS;''')

In [ ]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''CREATE TABLE COMMODITIES_RETURNS (
    DATE DATETIME,
	"CLOSE" NUMERIC,
    "Instrument" TEXT,
    "Return" NUMERIC
);''')


In [ ]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
#Add all Data at once:
c = conn.cursor()
c.executemany("INSERT INTO COMMODITIES_RETURNS(DATE, CLOSE, Instrument, Return) VALUES (?,?,?,?)", liste_SQL)    
conn.commit()


In [ ]:
#Select Data from SQL DB:   
c = conn.cursor()
c.execute("SELECT * FROM COMMODITIES_RETURNS")
new_data=c.fetchall()
ref_data = pd.DataFrame(new_data,columns=['DATE', 'CLOSE', 'Instrument', 'Return'])
ref_data = ref_data.set_index('DATE')
ref_data

In [ ]:
Comm_returns = ref_data.pivot(columns='Instrument', values=['Return'])
Comm_returns.to_excel('Commodities_returns.xlsx')
Comm_returns

In [ ]:
Comm_returns.describe()